In [74]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [106]:
# open classes.xlsx

classes_df = pd.read_excel('classes.xlsx', index_col=0)

classes_df[0:5]

,Category,Topic,Department,Course
0,Language,Intensive Elementary Chinese,CHI,103
1,Language,Advanced Spanish,SPA,108
2,Language,Beginner's French I,FRE,101
3,Language,Beginner's Russian I,RUS,101
4,Language,Elementary Korean I,KOR,101


In [120]:
# classes = classes_df['Class'].tolist()

schedule_info = {}

# for every row in the dataframe
for index, row in classes_df.iterrows():
    department = row['Department']
    course = row['Course']
    topic = row['Topic']

    search_URL = f"https://m.princeton.edu/default/courses/search?filter={topic}&search=Search&area={department}"

    # page = requests.get(search_URL)

    soup = BeautifulSoup(page.content, "html.parser")

    results = soup.find_all('a', class_='kgoui_list_item_action')

    if len(results) != 0:
        course_URL = f"https://m.princeton.edu{results[0].get('href')}"
    else:
        print(f'No results for {department} {course}')
        continue

    # page2 = requests.get(course_URL)

    soup = BeautifulSoup(page2.content, 'html.parser')

    results = soup.find(id='kgoui_Rcontent_I1_Rcontent_I0_Rsections')

    sections = results.find_all('div', class_='kgoui_object')

    section_info = []
    for section in sections:
        try: 
            section_info.append({'section': section.find('span', text=re.compile(r'Section')).text.replace('Section: ', ''),
            'schedule': section.find('span', text=re.compile(r'Schedule')).text.replace('Schedule: ', '')})
        except:
            continue
    
    schedule_info[f'{department} {course}'] = section_info

In [132]:
# create empty df2 with columns 'class', 'section', and 'schedule'
df2 = pd.DataFrame(columns=['Class', 'Section', 'Schedule'])

# for each key in schedule_info
for key in schedule_info:
    row = [key]
    for item in schedule_info[key]:
        row.append(item['Section'])
        row.append(item['Schedule'])

        # append row to df2
        df2.loc[len(df2)] = row
        row = [key]

# split schedule column of df2 into two by the first space.
df2[['Day', 'Time']] = df2['Schedule'].str.split(' ', 1, expand=True)

# drop schedule column
df2 = df2.drop(columns=['Schedule'])

df2

,class,section,day,time
0,CHI 103,C01,MTWThF,09:00 AM-09:50 AM
1,CHI 103,C02,MTWThF,10:00 AM-10:50 AM
2,CHI 103,C03,MTWThF,12:30 PM-01:20 PM
3,CHI 103,D01,F,02:30 PM-03:20 PM
4,CHI 103,D02,F,03:30 PM-04:20 PM
...,...,...,...,...
181,PSY 309,P13,W,11:00 AM-11:50 AM
182,PSY 309,P14,W,11:00 AM-11:50 AM
183,PSY 309,P15,W,01:30 PM-02:20 PM
184,PSY 309,P16,W,02:30 PM-03:20 PM


In [ ]:
time_pattern = r'(\d{2}:\d{2} [APM]{2})-(\d{2}:\d{2} [APM]{2})'
time_extract = df2['Time'].str.extract(time_pattern)

# Step 3: Convert the extracted strings into time objects
df2['Start'] = pd.to_datetime(time_extract[0], format='%I:%M %p').dt.time
df2['End'] = pd.to_datetime(time_extract[1], format='%I:%M %p').dt.time

# Step 4: Drop the original 'Time' column if needed
df2 = df2.drop(columns=['Time'])

In [ ]:
def parse_days(day_string):
    days = []
    if 'M' in day_string:
        days.append('Monday')
    if 'T' in day_string:
        days.append('Tuesday')
    if 'W' in day_string:
        days.append('Wednesday')
    if 'Th' in day_string:
        days.append('Thursday')
    if 'F' in day_string:
        days.append('Friday')
    return days

df2['Days'] = df2['Day'].apply(parse_days)